In [1]:
import pandas as pd
import numpy as np

from talkingdata import run
from models import PhoneBrandEstimator

%load_ext autoreload
%autoreload 2

In [2]:
gatrain = pd.read_csv('data/gender_age_train.csv')
pbrand = pd.read_csv('data/phone_brand_device_model.csv')

In [3]:
gatest = pd.read_csv('data/gender_age_test.csv')

In [4]:
pbrand.phone_brand.describe()

count     187245
unique       131
top           小米
freq       43210
Name: phone_brand, dtype: object

In [5]:
gaptrain = gatrain.merge(pbrand.drop_duplicates('device_id',keep='first'), on='device_id')

In [6]:
gaptrain.columns

Index(['device_id', 'gender', 'age', 'group', 'phone_brand', 'device_model'], dtype='object')

In [7]:
translations = pd.read_csv('data/phone_brand_chinese_to_english.csv', sep=',')

In [8]:
translations.head()

,ch,eng
0,三星,samsung
1,天语,Ktouch
2,海信,hisense
3,联想,lenovo
4,欧比,obi


In [9]:
translations.ch.describe()

count     55
unique    55
top       梦米
freq       1
Name: ch, dtype: object

Como podemos ver, hay muchos menos modelos con traducciones (55) que el total (131)

In [10]:
for chb, engb in zip(translations['ch'], translations['eng']):
    gaptrain.replace(to_replace=chb, value=engb, inplace=True)

#gaptrain.replace(to_replace=translations['ch'], value=translations['eng'], inplace=True)

Acabo de descubrir que 小米 es la traduccion de Xiaomi. Y 红米 es Red Rice

In [11]:
gaptrain.replace(to_replace=['小米', '红米'], value=['Xiaomi', 'Red Rice'], inplace=True)

In [12]:
gaptrain.device_id = gaptrain.device_id.astype('category')
gaptrain.device_model = gaptrain.device_model.astype('category')
gaptrain.phone_brand = gaptrain.phone_brand.astype('category')

Ahora vamos a predecir sobre el test set.

In [16]:
gaptest = gatest.merge(pbrand.drop_duplicates('device_id',keep='first'), on='device_id')

for chb, engb in zip(translations['ch'], translations['eng']):
    gaptest.replace(to_replace=chb, value=engb, inplace=True)
gaptest.replace(to_replace=['小米', '红米'], value=['Xiaomi', 'Red Rice'], inplace=True)

In [18]:
dataset = gaptrain.drop(['gender', 'age'], axis=1)

In [17]:
# Leak
dataset['row'] = dataset.index
gaptest['row'] = gaptest.index

In [19]:
dataset.head(10)

,device_id,group,phone_brand,device_model,row
0,-8076087639492063270,M32-38,Xiaomi,MI 2,0
1,-2897161552818060146,M32-38,Xiaomi,MI 2,1
2,-8260683887967679142,M32-38,Xiaomi,MI 2,2
3,-4938849341048082022,M29-31,Xiaomi,红米note,3
4,245133531816851882,M29-31,Xiaomi,MI 3,4
5,-1297074871525174196,F24-26,OPPO,R1C,5
6,236877999787307864,M32-38,酷派,F2,6
7,-8098239495777311881,M32-38,Xiaomi,红米note,7
8,176515041953473526,M32-38,vivo,X3T,8
9,1596610250680140042,F33-42,samsung,Galaxy S4,9


In [21]:
results, rpred = run(PhoneBrandEstimator, dataset.dropna(), valid=gaptest)
pred = pd.DataFrame(rpred[:len(rpred)/4], columns=results.columns[2:])
pred.insert(0, 'device_id', gatest.device_id)
pred.to_csv('data/valid_predictions_lrlk.csv', index=False)

Predicted LogLoss: 0_ 2.40066
Predicted LogLoss: 1_ 2.40196
Predicted LogLoss: 2_ 2.40330
Predicted LogLoss: 3_ 2.39789


/home/gonzalo/virtualenvs/machine-learning/lib/python3.4/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


In [22]:
len(pred)

112071